# Project Name: FHR and UC Analysis

### Generate Data

In [ ]:
import pandas as pd
import numpy as np

# Set parameters for dataset generation
num_rows = 1000  # Number of rows in the dataset
num_columns = 3  # Number of columns (including time, FHR, and UC)

# Simulate time in milliseconds (each row is 250 milliseconds apart)
time_column = np.arange(250, 250 * (num_rows + 1), 250)

# Simulate FHR (beats per minute), random values around a typical range for fetal heart rate
fhr_column = np.random.randint(120, 160, size=num_rows)

# Simulate UC (TOCO), random values representing uterine contractions
uc_column = np.random.randint(5, 20, size=num_rows)

# Generate additional random columns to reach 10 columns
extra_columns = {
    f"Extra_Col_{i+1}": np.random.random(size=num_rows) * 100 for i in range(num_columns - 3)
}

# Combine all into a DataFrame
data = {
    "Time (ms)": time_column,
    "FHR (bpm)": fhr_column,
    "UC (TOCO)": uc_column
}

df = pd.DataFrame(data)
df

### Plotting Graphs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert time from milliseconds to seconds
df['Time (s)'] = df['Time (ms)'] / 1000

# Plot FHR vs Time
plt.figure(figsize=(10, 5))
plt.plot(df['Time (s)'], df['FHR (bpm)'], label='FHR (bpm)', color='blue')
plt.title('Fetal Heart Rate (FHR) over Time')
plt.xlabel('Time (s)')
plt.ylabel('FHR (bpm)')
plt.legend()
plt.show()

# Plot UC vs Time
plt.figure(figsize=(10, 5))
plt.plot(df['Time (s)'], df['UC (TOCO)'], label='UC (TOCO)', color='red')
plt.title('Uterine Contractions (UC) over Time')
plt.xlabel('Time (s)')
plt.ylabel('UC (TOCO)')
plt.legend()
plt.show()

### FHR Analysis (Epochs)

In [ ]:
# Define function to calculate pulse interval from bpm
def bpm_to_pulse_interval(bpm):
    return (60 * 1000) / bpm  # Pulse interval in milliseconds

# Resampling to 1-minute chunks (Assuming the time is continuous)
df['Minute'] = df['Time (s)'] // 60

# Epoch analysis: Divide each minute into 16 epochs (3.75 seconds each = 15 data points)
epoch_size = 15  # 3.75 seconds * 4 data points per second = 15

fhr_epoch_avg = []
pulse_intervals = []

for minute in df['Minute'].unique():
    minute_data = df[df['Minute'] == minute]
    
    for epoch in range(0, len(minute_data), epoch_size):
        epoch_data = minute_data.iloc[epoch:epoch + epoch_size]
        
        # Calculate average FHR
        avg_fhr = epoch_data['FHR (bpm)'].mean()
        fhr_epoch_avg.append(avg_fhr)
        
        # Calculate pulse interval
        avg_pulse_interval = bpm_to_pulse_interval(avg_fhr)
        pulse_intervals.append(avg_pulse_interval)

# Convert results to DataFrame for easier analysis
epoch_df = pd.DataFrame({
    'Epoch': range(len(fhr_epoch_avg)),
    'Avg FHR (bpm)': fhr_epoch_avg,
    'Avg Pulse Interval (ms)': pulse_intervals
})

print(epoch_df.head())

### UC Peak Detection

In [ ]:
from scipy.signal import find_peaks, peak_widths

# UC peak detection
uc_values = df['UC (TOCO)'].values
peaks, _ = find_peaks(uc_values, height=0)  # You can set a height threshold if needed

# Calculate the width of peaks at half their maximum height
widths = peak_widths(uc_values, peaks, rel_height=0.5)

# Convert widths from data points to seconds (4 data points per second)
widths_in_seconds = widths[0] / 4

# Count peaks where the width is greater than 30 seconds
wide_peaks = widths_in_seconds[widths_in_seconds > 30]

# Calculate the average duration of these peaks
avg_duration = wide_peaks.mean()

print(f"Number of peaks wider than 30 seconds: {len(wide_peaks)}")
print(f"Average duration of wide peaks: {avg_duration} seconds")